<a href="https://colab.research.google.com/github/sarikasea/Agentic-RAG-with-Llamaindex/blob/main/L1_Router_Engine_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 1: Router Engine

Welcome to Lesson 1.

To access the `requirements.txt` file, the data/pdf file required for this lesson and the `helper` and `utils` modules, please go to the `File` menu and select`Open...`.

I hope you enjoy this course!

## Setup

In [ ]:
%pip install --upgrade python-dotenv
%pip install --upgrade llama-index
#%pip install --upgrade llama-index-llms-openai
%pip install --upgrade llama-index-embeddings-openai
%pip install llama-index-llms-gemini llama-index-embeddings-gemini
%pip install --upgrade llama-index-llms-google-genai llama-index-embeddings-google-genai

In [ ]:
#from helper import get_openai_api_key

#OPENAI_API_KEY = get_openai_api_key()

In [ ]:
from google.colab import userdata
import os

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

## Load Data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

**Note**: The pdf file is included with this lesson. To access it, go to the `File` menu and select`Open...`.

In [ ]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()

### Define LLM and Embedding model

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

Let's switch to the recommended `llama-index-llms-google-genai` library to initialize the LLM and embedding model with Gemini.

In [ ]:
from llama_index.core import Settings
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbedding

# Initialize the LLM and embedding model with Gemini
llm = Gemini(model="gemini-2.0-flash-exp", temperature=0.1)
embed_model = GeminiEmbedding(model="models/embedding-001")

# Set them in Settings
Settings.llm = llm
Settings.embed_model = embed_model

/tmp/ipython-input-991455480.py:6: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/)
  llm = Gemini(model="gemini-2.0-flash-exp", temperature=0.1)
/tmp/ipython-input-991455480.py:7: DeprecationWarning: Call to deprecated class GeminiEmbedding. (Should use `llama-index-embeddings-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/embeddings/google_genai/)
  embed_model = GeminiEmbedding(model="models/embedding-001")


## Define Summary Index and Vector Index over the Same Data

In [ ]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

To use Gemini, you need to set up the API key in the Colab secrets manager and configure the LlamaIndex settings to use the Gemini model and embedding.

In [ ]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

## Define Query Engines and Set Metadata

In [ ]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [ ]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper."
    ),
)

## Define Router Query Engine

In [ ]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [ ]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: The question asks for a summary of the document, and choice 1 explicitly states it is useful for summarization questions..
MetaGPT is introduced as a meta-programming framework designed for multi-agent collaboration using large language models (LLMs). It incorporates Standardized Operating Procedures (SOPs) into prompt sequences, streamlining workflows and enabling agents with domain expertise to verify intermediate results, thus reducing errors. By using an assembly line paradigm, MetaGPT assigns diverse roles to agents, breaking down complex tasks into subtasks that require collaboration. The framework's effectiveness is demonstrated through collaborative software engineering benchmarks, where it generates more coherent solutions compared to previous chat-based multi-agent systems. The design of MetaGPT improves robustness, reduces unproductive collaboration among LLM-based agents, and elevates code generation quality through a novel executive feedback mecha

In [ ]:
print(len(response.source_nodes))

34


In [ ]:
response = query_engine.query(
    "How do agents share information with other agents?"
)
print(str(response))

Selecting query engine 1: The question 'How do agents share information with other agents?' requires retrieving specific details about the MetaGPT framework's implementation. Choice 2, 'Useful for retrieving specific context from the MetaGPT paper,' is therefore the most relevant..
Agents share and comply with development and communication protocols.



In [ ]:
response = query_engine.query(
    "How many sections are in the paper, and provide a 5 sentence summary of each"
)
print(str(response))

Selecting query engine 1: The question requires retrieving specific information (number of sections) and summarizing each section from the MetaGPT paper. Choice 2 is specifically designed for retrieving specific context from the paper, which is necessary to answer the question..
There are two sections provided in the context.

The first section discusses program call flow for recommendation engine development, emphasizing effective communication in software design scenarios and standard operating procedures.

The second section includes a product requirement document that lists task files such as "main.py", "canvas.py", and others. It specifies that the Tkinter library is used for the GUI and Pillow for image file handling, with "main.py" as the application's entry point. The project requirements and implementation approach are clear. It also addresses potential team familiarity with Tkinter and Pillow libraries, suggesting a possible need for learning time before development.



## Let's put everything together

In [ ]:
from utils import get_router_query_engine

query_engine = get_router_query_engine("metagpt.pdf")

AuthenticationError: Error code: 401 - {'error': {'message': "You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

Here is the updated `get_router_query_engine` function code. Please copy and paste this code into your `utils.py` file, replacing the existing function definition.

In [ ]:
from utilsgemini import get_router_query_engine

query_engine = get_router_query_engine("metagpt.pdf")

ImportError: cannot import name 'GoogleGenerativeAI' from 'llama_index.llms' (unknown location)

In [ ]:
response = query_engine.query("Tell me about the ablation study results?")
print(str(response))